In [1]:
import tensorflow as tf
import numpy as np
import random as rd
from keras.utils import np_utils
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
image = np.loadtxt('bbs-train.txt')
label = np.loadtxt('label-train.txt')
# image /= np.max(image_train)

In [3]:
# initialize the dataset
image_train = image[:4500]
image_val = image[4500:]
labels = np.zeros((len(label),2))
for i in range(len(label)):
    if label[i,1] == 0:
        labels[i] = [0.0, 1.0]
    if label[i,1] == 1:
        labels[i] = [1.0, 0.0]

label_train = labels[:4500]
label_val = labels[4500:]
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [4]:
t = label[0:,1:]
t_x = np.reshape(t, (1,4546))
labels = np_utils.to_categorical(t_x, 2)

In [5]:
learning_rate = 0.05
epochs = 10
batch_size = 100

x = tf.placeholder(tf.float32,[None, 800])
y = tf.placeholder(tf.float32, [None, 2])

# hidden layer1 connect with the input layer
W1 = tf.Variable(tf.zeros([800, 500]))
b1 = tf.Variable(tf.zeros([500]))

# hidden layer2 connect with the output layer
W2 = tf.Variable(tf.zeros([500,2]))
b2 = tf.Variable(tf.zeros([2]))

hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

#output layer
y_ = tf.nn.softmax(tf.add(tf.matmul(hidden_out, W2), b2))
#to make sure that we never get a case were we have a log(0) operation occurring during training
y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)

cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped) + (1 - y) * tf.log(1 - y_clipped), axis=1))
# add an optimiser
optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

# finally setup the initialisation operator
init_op = tf.global_variables_initializer()

# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [6]:
with tf.Session() as sess:
    # initialise the variables
    sess.run(init_op)
    total_batch = int(len(label_train) / batch_size)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = next_batch(batch_size, image_train, label_train)
            _, c = sess.run([optimiser, cross_entropy], 
                          feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
        
    print()
    print("training complete! The accurancy is: ")
    print(sess.run(accuracy, feed_dict={x: image_val, y: label_val}))
        

Epoch: 1 cost = 1.292
Epoch: 2 cost = 1.261
Epoch: 3 cost = 1.261
Epoch: 4 cost = 1.271
Epoch: 5 cost = 1.242
Epoch: 6 cost = 1.255
Epoch: 7 cost = 1.272
Epoch: 8 cost = 1.271
Epoch: 9 cost = 1.258
Epoch: 10 cost = 1.263

training complete! The accurancy is: 
0.913043
